In [1]:
import hickle as hkl
import numpy as np

from lib.read_data import u_desvio as u_true
from lib.read_data import y_desvio as y_true

ny = y_true.shape[1]
nu = u_true.shape[1]
theta = hkl.load("../export/ARX.hkl")

A = theta[:ny, :].T
B = theta[ny:, :].T


def model(y: np.ndarray, u: np.ndarray):
    if y.shape != (ny,):
        raise ValueError(f"y inválido: shape {y.shape}, esperado {(ny,)}")

    if u.shape != (nu,):
        raise ValueError(f"u inválido: shape {u.shape}, esperado {(nu,)}")

    return A @ y + B @ u  # y(k+1)


# Simulando malha aberta


In [2]:
ncycles = 150
cycles = np.arange(0, ncycles)

# Prealocando histórico de saída e entrada
y = np.zeros((ncycles, ny))
y[0] = y_true[0]

u = np.zeros((ncycles, nu))
u[:] = np.array([-70, -45, 5, 25])
u[0] = u_true[0]


for k in range(1, ncycles):
    # Atualiza o estado
    y[k] = model(y[k - 1], u[k])


In [3]:
from lib.read_data import u_ref, y_ref

# u_ref = np.zeros_like(u_ref)
# y_ref = np.zeros_like(y_ref)

y = y + np.tile(y_ref, (y.shape[0], 1))
u = u + np.tile(u_ref, (u.shape[0], 1))


In [4]:
# Plotando gráfico
from lib.plot import colors, legend_outside, plt, u_labels, y_labels

_, axes = plt.subplots(2, 1, sharex=True, figsize=(10, 7))

# Entradas
for i in range(len(u_labels)):
    axes[0].plot(cycles, u[:, i], label=u_labels[i])

axes[0].set_ylabel("Duração da etapa / s")

# Saídas
for i in range(len(y_labels)):
    axes[1].plot(cycles, y[:, i], label=y_labels[i], color=colors[i])

axes[1].set_ylabel("KPIs / %")

axes[-1].set_xlabel("Ciclo")
for ax in axes:
    legend_outside(ax)
    ax.grid(True)

plt.tight_layout()
plt.savefig("../figures/simulate.png")
plt.close()
